In [1]:
# KERNEL: mlenv

# Import dependencies
import warnings
warnings.filterwarnings("ignore")

In [2]:
#dependencies
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.model_selection import train_test_split

In [21]:
# Data Load
all_star_df = pd.read_csv("../Queries/Testing_Files/company_all_star.csv")
all_star_df.head(30)

,ticker,date,company_name,url,employee_count,revenue,sector,city_name,state_name,country,latitude,longitude,open_val,high_val,low_val,close_val,volume,volume_weight,number_of_transactions,percent_change
0,AMD,2020-03-12,Advanced Micro Devices Inc,{url:amd.com},5k-10k,over-1b,TBD,Santa Clara,CA,US,37.233325,-121.684635,42.200,43.9100,39.600,43.90,86689681.0,41.6701,381223,4.028436
1,AMD,2020-03-15,Advanced Micro Devices Inc,{url:amd.com},5k-10k,over-1b,TBD,Santa Clara,CA,US,37.233325,-121.684635,39.080,43.3700,38.510,38.71,84545868.0,41.0812,374962,0.946776
2,AMD,2020-03-16,Advanced Micro Devices Inc,{url:amd.com},5k-10k,over-1b,TBD,Santa Clara,CA,US,37.233325,-121.684635,40.190,42.8800,38.300,41.88,92741881.0,41.1240,434519,4.205026
3,AMD,2020-03-17,Advanced Micro Devices Inc,{url:amd.com},5k-10k,over-1b,TBD,Santa Clara,CA,US,37.233325,-121.684635,39.540,41.9500,36.750,39.12,106949287.0,39.6363,591862,1.062215
4,AMD,2020-03-18,Advanced Micro Devices Inc,{url:amd.com},5k-10k,over-1b,TBD,Santa Clara,CA,US,37.233325,-121.684635,39.560,41.7000,37.690,39.82,88939024.0,40.2337,396388,0.657230
5,AMD,2020-03-19,Advanced Micro Devices Inc,{url:amd.com},5k-10k,over-1b,TBD,Santa Clara,CA,US,37.233325,-121.684635,41.510,42.4825,39.580,39.61,106859502.0,41.2299,533411,4.577210
6,AMD,2020-03-22,Advanced Micro Devices Inc,{url:amd.com},5k-10k,over-1b,TBD,Santa Clara,CA,US,37.233325,-121.684635,40.620,42.3200,38.950,41.64,101704663.0,40.7188,493186,2.511078
7,AMD,2020-03-23,Advanced Micro Devices Inc,{url:amd.com},5k-10k,over-1b,TBD,Santa Clara,CA,US,37.233325,-121.684635,44.040,46.8100,43.990,46.22,106794151.0,45.7175,535460,4.950045
8,AMD,2020-03-24,Advanced Micro Devices Inc,{url:amd.com},5k-10k,over-1b,TBD,Santa Clara,CA,US,37.233325,-121.684635,46.790,47.8750,44.425,44.63,93760389.0,46.5954,488900,4.616371
9,AMD,2020-03-25,Advanced Micro Devices Inc,{url:amd.com},5k-10k,over-1b,TBD,Santa Clara,CA,US,37.233325,-121.684635,45.780,47.5000,45.400,47.50,73915608.0,46.7201,379715,3.757099


In [4]:
# check dtypes
all_star_df.dtypes

ticker                     object
date                       object
company_name               object
url                        object
employee_count             object
revenue                    object
sector                     object
city_name                  object
state_name                 object
country                    object
latitude                  float64
longitude                 float64
open_val                  float64
high_val                  float64
low_val                   float64
close_val                 float64
volume                    float64
volume_weight             float64
number_of_transactions      int64
percent_change            float64
dtype: object

In [5]:
# training model needs to be "int64" for it to be fit, converted all float64s
all_star_df[[
    "latitude", 
    "longitude", 
    "open_val", 
    "high_val", 
    "low_val", 
    "close_val", 
    "volume",
    "volume_weight",
    "percent_change"]] = all_star_df[[
                                "latitude", 
                                "longitude",
                                "open_val", 
                                "high_val", 
                                "low_val", 
                                "close_val", 
                                "volume",
                                "volume_weight",
                                "percent_change"
                                ]].astype("int")

In [6]:
# verify .astype() changes
all_star_df.dtypes

ticker                    object
date                      object
company_name              object
url                       object
employee_count            object
revenue                   object
sector                    object
city_name                 object
state_name                object
country                   object
latitude                   int64
longitude                  int64
open_val                   int64
high_val                   int64
low_val                    int64
close_val                  int64
volume                     int64
volume_weight              int64
number_of_transactions     int64
percent_change             int64
dtype: object

In [7]:
# need help with which features are to be used and which to drop

In [8]:
# Create our features 
X = all_star_df.drop("ticker", axis=1)
X = pd.get_dummies(X)

# Target 
y = all_star_df["ticker"]

In [9]:
# One dataset for geolocation and another for stock behaviour? run each through the model?

In [10]:
X.describe()

,latitude,longitude,open_val,high_val,low_val,close_val,volume,volume_weight,number_of_transactions,percent_change,...,state_name_WA,state_name_WI,country_Argentina,country_Australia,country_CA,country_CH,country_CN,country_Netherlands,country_UK,country_US
count,50874.000000,50874.000000,50874.000000,50874.000000,50874.000000,50874.000000,5.087400e+04,50874.000000,5.087400e+04,50874.000000,...,50874.000000,50874.000000,50874.000000,50874.000000,50874.000000,50874.000000,50874.000000,50874.000000,50874.000000,50874.000000
mean,37.089063,-90.437846,296.988167,301.174765,292.553426,296.896765,7.683480e+06,296.901010,7.490834e+04,1.066222,...,0.069348,0.009907,0.009907,0.009907,0.009907,0.019814,0.009907,0.019814,0.009907,0.910839
std,11.113165,51.767554,480.529339,486.203403,474.318435,480.192698,1.728108e+07,480.303661,1.334121e+05,1.564538,...,0.254047,0.099040,0.099040,0.099040,0.099040,0.139361,0.099040,0.139361,0.099040,0.284979
min,-34.000000,-122.000000,4.000000,4.000000,4.000000,4.000000,6.954300e+04,4.000000,2.794000e+03,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,37.000000,-121.000000,82.000000,84.000000,81.000000,82.000000,1.261210e+06,82.000000,2.208600e+04,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,37.000000,-111.000000,155.000000,158.000000,152.000000,155.000000,2.626642e+06,155.000000,3.648000e+04,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,41.000000,-77.000000,302.000000,308.000000,298.000000,302.000000,6.403628e+06,302.000000,6.899325e+04,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,52.000000,150.000000,3744.000000,3773.000000,3696.000000,3731.000000,4.016934e+08,3722.000000,2.966979e+06,40.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [11]:
y.value_counts()

AMD     504
MRNA    504
ORLY    504
ODFL    504
OKTA    504
       ... 
COST    504
ZS      504
BIIB    503
ABNB    315
LCID    160
Name: ticker, Length: 102, dtype: int64

In [20]:
np.unique(y)

array(['AAPL', 'ABNB', 'ADBE', 'ADI', 'ADP', 'ADSK', 'AEP', 'ALGN',
       'AMAT', 'AMD', 'AMGN', 'AMZN', 'ANSS', 'ASML', 'ATVI', 'AVGO',
       'AZN', 'BIDU', 'BIIB', 'BKNG', 'BMRN', 'CDNS', 'CHTR', 'CMCSA',
       'COST', 'CPRT', 'CRWD', 'CSCO', 'CSX', 'CTAS', 'CTSH', 'DDOG',
       'DLTR', 'DOCU', 'DXCM', 'EA', 'EBAY', 'EXC', 'FAST', 'FB', 'FISV',
       'FTNT', 'GILD', 'GOOG', 'GOOGL', 'HON', 'IDXX', 'ILMN', 'INTC',
       'INTU', 'ISRG', 'JD', 'KDP', 'KHC', 'KLAC', 'LCID', 'LRCX', 'LULU',
       'MAR', 'MCHP', 'MDLZ', 'MELI', 'MNST', 'MRNA', 'MRVL', 'MSFT',
       'MTCH', 'MU', 'NFLX', 'NTES', 'NVDA', 'NXPI', 'ODFL', 'OKTA',
       'ORLY', 'PANW', 'PAYX', 'PCAR', 'PDD', 'PEP', 'PYPL', 'QCOM',
       'REGN', 'ROST', 'SBUX', 'SGEN', 'SIRI', 'SNPS', 'SPLK', 'SWKS',
       'TEAM', 'TMUS', 'TSLA', 'TXN', 'VRSK', 'VRSN', 'VRTX', 'WBA',
       'WDAY', 'XEL', 'ZM', 'ZS'], dtype=object)

In [12]:
# train the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [13]:
# random forest classfier
# n_estimator default at 100
random_forest = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
random_forest.fit(X_train, y_train)

BalancedRandomForestClassifier(random_state=1)

In [14]:
# confusion matrix
# this matrix is gigantic
y_pred = random_forest.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[136,   0,   0, ...,   0,   0,   0],
       [  0,  84,   0, ...,   0,   0,   0],
       [  0,   0, 140, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ..., 121,   0,   0],
       [  0,   0,   0, ...,   0, 107,   0],
       [  0,   0,   0, ...,   0,   0, 139]])

In [15]:
balanced_accuracy_score(y_test, y_pred)

1.0

In [16]:
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

       AAPL       1.00      1.00      1.00      1.00      1.00      1.00       136
       ABNB       1.00      1.00      1.00      1.00      1.00      1.00        84
       ADBE       1.00      1.00      1.00      1.00      1.00      1.00       140
        ADI       1.00      1.00      1.00      1.00      1.00      1.00       121
        ADP       1.00      1.00      1.00      1.00      1.00      1.00       123
       ADSK       1.00      1.00      1.00      1.00      1.00      1.00       143
        AEP       1.00      1.00      1.00      1.00      1.00      1.00       145
       ALGN       1.00      1.00      1.00      1.00      1.00      1.00       115
       AMAT       1.00      1.00      1.00      1.00      1.00      1.00       138
        AMD       1.00      1.00      1.00      1.00      1.00      1.00       126
       AMGN       1.00      1.00      1.00      1.00      1.00      1.00       128
   

In [17]:
sorted(zip(X.columns, random_forest.feature_importances_), reverse=True)

[('volume_weight', 0.011370306285222979),
 ('volume', 0.007367872631593425),
 ('url_{url:zscaler.com}', 0.0039177909440596004),
 ('url_{url:zoom.us}', 0.0031232153746737555),
 ('url_{url:xfinity.com}', 0.0021037297155626234),
 ('url_{url:workday.com}', 0.002935956063030557),
 ('url_{url:walgreensbootsalliance.com}', 0.003039891079384922),
 ('url_{url:vrtx.com}', 0.002152223646733372),
 ('url_{url:verisk.com}', 0.0028072536387623755),
 ('url_{url:verisign.com}', 0.002237173223302315),
 ('url_{url:ti.com}', 0.004019063335416444),
 ('url_{url:tesla.com}', 0.0031639251434430566),
 ('url_{url:t-mobile.com}', 0.002474492408334659),
 ('url_{url:synopsys.com}', 0.003774564856789063),
 ('url_{url:starbucks.com}', 0.0028968710703392297),
 ('url_{url:splunk.com}', 0.0034754138040056885),
 ('url_{url:spectrum.com}', 0.0024020068854212866),
 ('url_{url:skyworksinc.com}', 0.0029983637729756973),
 ('url_{url:siriusxm.com}', 0.00181165638797171),
 ('url_{url:seagen.com}', 0.003098607062037438),
 ('url